In [12]:
import pandas as pd

In [13]:
import os

In [14]:
list_of_files = os.listdir()


In [15]:
list_of_excel_files = [e for e in  list_of_files if '.xlsx' in e and 'updated' not in e]


In [16]:
list_of_excel_files

['31-Dec-21_scraped_data.xlsx',
 '05-Jan-22_scraped_data.xlsx',
 '01-Jan-22_scraped_data.xlsx',
 '04-Jan-22_scraped_data.xlsx',
 '06-Jan-22_scraped_data.xlsx',
 '02-Jan-22_scraped_data.xlsx',
 '03-Jan-22_scraped_data.xlsx',
 '07-Jan-22_scraped_data.xlsx']

In [17]:
dates = [e[:9] for e in list_of_excel_files]
dates

['31-Dec-21',
 '05-Jan-22',
 '01-Jan-22',
 '04-Jan-22',
 '06-Jan-22',
 '02-Jan-22',
 '03-Jan-22',
 '07-Jan-22']

In [18]:
import datetime
import numpy as np
# create a dictionary of dates in the list of files
formatted_dates_dict = {}
for date in dates:
    d = datetime.datetime.strptime(date,'%d-%b-%y').strftime('%Y-%m-%d')
    formatted_dates_dict[d] = date

   

In [19]:
#Function to convert text to float as values are stored as text when scraping
def convert_to_float(x):
    try:
        return float(x.replace(',',''))
    except:
        return np.nan

In [20]:
a = []
for i in range(6):
    a.append(i+1)
    a.append(i+3)
    
    

sorted_dates = sorted(formatted_dates_dict.keys())
#Go through each date
for i,d in enumerate(sorted_dates):
    if i < len(sorted_dates)-1:
        yesterday = formatted_dates_dict[sorted_dates[i]]
        today = formatted_dates_dict[sorted_dates[i+1]]
        ydf = pd.read_excel(f'{yesterday}_scraped_data.xlsx')
        tdf = pd.read_excel(f'{today}_scraped_data.xlsx')
#ydf = yesterday df
#tdf = today df        
        
        ydf.columns = [col+'_x' for col in ydf.columns]
#added x to the column names of yeterday df's             
        tdf_merged = tdf.merge(ydf,left_on='crypto',right_on = 'crypto_x')
#merged on the crypto url which is unique            
        

        six_months  = []
        for col in tdf.columns[1:19]:
            #print(col)
            month = col.split("_")[0]
            if month not in six_months:
                six_months.append(month)

#performing the statistics
        for month in six_months:
            try:
                tdf_merged[f'{month}_votes'] = tdf_merged[f'{month}_votes'].apply(lambda x: convert_to_float(x))
                tdf_merged[f'{month}_mean'] = tdf_merged[f'{month}_mean'].apply(lambda x: convert_to_float(x))
                tdf_merged[f'{month}_votes_x'] = tdf_merged[f'{month}_votes_x'].apply(lambda x: convert_to_float(x))
                tdf_merged[f'{month}_mean_x'] = tdf_merged[f'{month}_mean_x'].apply(lambda x: convert_to_float(x))    


                tdf_merged[f'{month}_newVotes_today'] =   tdf_merged[f'{month}_votes'] - tdf_merged[f'{month}_votes_x']
                tdf_merged[f'{month}_dailyNetValue'] =   ((tdf_merged[f'{month}_votes'] * tdf_merged[f'{month}_mean']) - (tdf_merged[f'{month}_mean_x'] * tdf_merged[f'{month}_votes_x']))/(tdf_merged[f'{month}_votes'] - tdf_merged[f'{month}_votes_x'])

            except:
                tdf_merged[f'{month}_newVotes_today'] =   "There are not enough estimates to calculate the stats"
                tdf_merged[f'{month}_dailyNetValue'] =   "There are not enough estimates to calculate the stats"
                pass
                
        for col in tdf_merged:
            if col.split("_")[-1]=='x':
                tdf_merged.drop(columns = col,inplace=True)
                
        tdf_merged.to_excel(f"{today}_updated.xlsx",index=False)
        
    